In [11]:
import gymnasium as gym
import torch
import torch.nn as nn
from stable_baselines3 import A2C
from stable_baselines3.common.policies import ActorCriticCnnPolicy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import gym_BinPack3D
from gym_BinPack3D.envs import Box, Rotate
from stable_baselines3.common.env_checker import check_env
from matplotlib import pyplot as plt
import numpy as np
# %matplotlib notebook



In [12]:
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(CustomNetwork, self).__init__(observation_space, features_dim)

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        self.fc = nn.Sequitial(
            nn.Linear(64*8*8, features_dim),
            nn.ReLU(),
            nn.Linear(256, 2)
        )

    def forward(self, observations):
        hmap = observations["height_map"]
        coming_boxes = observations["coming_boxes"]
        valid_placement_mask = observations["valid_placement_mask"]

        hmap = self.cnn(hmap)
        coming_boxes = coming_boxes.flatten()
        valid_placement_mask = valid_placement_mask.flatten()

        x = torch.cat([hmap, coming_boxes, valid_placement_mask], dim=1)
        x = self.fc(x)

        return x
    
class CustomPolicy(ActorCriticCnnPolicy):
    def __init__(self, observation_space, action_space, lr_schedule, net_arch=None, activation_fn=nn.Tanh,
                 *args, **kwargs):
        super(CustomPolicy, self).__init__(observation_space, action_space, lr_schedule,
                                           net_arch=net_arch, activation_fn=activation_fn,
                                           *args, **kwargs)

        self.features_extractor = CustomNetwork(observation_space)
        
env = gym.make('BinPack3D-v0', 
                container_size = (10, 12, 20),
                boxSeqGenerator='random', 
                enabled_rotations = [Rotate.NOOP],
                n_foreseeable_box = 1,
                minSideLen = 2,
                maxSideLen = 5,
            )

class FlattenObservation(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(np.product(env.observation_space.shape),))

    def observation(self, observation):
        return np.concatenate([space.flatten() for space in observation.values()])

# Wrap your environment
env = FlattenObservation(env)


# Use the custom policy
model = A2C(CustomPolicy, env, verbose=1)

using random box sequence
Box to be sampled:
Box: Size 1 1 1 Position 0 0 0
Box: Size 2 3 4 Position 0 0 0


c:\Users\bente\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (10, 12)
  logger.warn(


AssertionError: Expect all shape elements to be an integer, actual type: (<class 'NoneType'>,)